In [1]:
import copy
import json
import math
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
%matplotlib inline

In [2]:
with open('input19', 'r') as f:
    lines = [l.strip() for l in f.readlines()]

## Part 1

In [3]:
def get_24_rotations():
    rollmat = np.array([[1, 0, 0], [0, 0, 1], [0, -1, 0]], dtype=float)
    turnmat = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]], dtype=float)
    rotations = []
    current_rot = np.eye(3)
    for cycle in range(2):
        for step in range(3): # RTTT 3 times
            current_rot = rollmat @ current_rot 
            rotations.append(current_rot) # R
            for i in range(3): # TTT
                current_rot = turnmat @ current_rot
                rotations.append(current_rot)
        current_rot = rollmat @ turnmat @ rollmat @ current_rot # RTR
    return rotations

In [4]:
def rotate_point(point, matrix):
    result = np.matmul(matrix, np.array([[point[0]], [point[1]], [point[2]]]))
    return result.flatten().astype(int).tolist()

In [5]:
def offset_point(point, offset):
    return [point[0] - offset[0], point[1] - offset[1], point[2] - offset[2]]

In [6]:
rotation_matrices = get_24_rotations()

In [7]:
def find_position_rotation(scanners, index1, index2):
    correct_rotation = None
    correct_matches = None
    for base_point1 in scanners[index1]:
        for base_point2 in scanners[index2]:
            for matrix in rotation_matrices:
                points1 = [offset_point(p, base_point1) for p in scanners[index1]]
                points2 = [rotate_point(offset_point(p, base_point2), matrix) for p in scanners[index2]]
                matching_points = []
                matching_point_indices = []
                for i, point in enumerate(points1):
                    for j, other in enumerate(points2):
                        if point[0] == other[0] and point[1] == other[1] and point[2] == other[2]:
                            matching_points.append(point)
                            matching_point_indices.append((i, j))
                if len(matching_points) >= 12:
                    correct_rotation = matrix
                    correct_matches = matching_point_indices
    if not correct_matches:
        return None, None
    p1 = scanners[index1][correct_matches[0][0]]
    p2 = rotate_point(scanners[index2][correct_matches[0][1]], correct_rotation)
    position = [p1[0] - p2[0], p1[1] - p2[1], p1[2] - p2[2]]
    return correct_rotation, position

In [9]:
scanners = []
for line in lines:
    if line.startswith('---'):
        scanners.append([])
    elif len(line) > 0:
        scanners[-1].append([int(v) for v in line.split(',')])

scanner_positions = [None] * len(scanners)
scanner_positions[0] = [0, 0, 0]
scanner_rotations = [None] * len(scanners)
scanner_rotations[0] = np.eye(3)

while sum(p is None for p in scanner_positions) > 0:
    found = False
    for i in [i for i, p in enumerate(scanner_positions) if p is None]:
        if found:
            break
        for j in [j for j, p in enumerate(scanner_positions) if p is not None]:
            rot, pos = find_position_rotation(scanners, j, i)
            if pos is not None:
                print(f'Found scanner {i} from {j}: ({pos[0]}, {pos[1]}, {pos[2]})')
                scanner_positions[i] = pos
                scanner_rotations[i] = rot
                new_scanner = []
                for p in scanners[i]:
                    new_scanner.append(offset_point(rotate_point(p, rot), [-p for p in pos]))
                scanners[i] = new_scanner
                found = True
                break
    if not found:
        print('Failed to find next scanner')
        break

beacons = set()
for scanner in scanners:
    for p in scanner:
        beacons.add(tuple(p))
print(f'Found {len(beacons)} beacons')

Found scanner 13 from 0: (1119, -64, -35)
Found scanner 9 from 13: (1144, -108, -1182)
Found scanner 7 from 9: (1266, 1249, -1241)
Found scanner 14 from 0: (-73, -1233, 97)
Found scanner 16 from 13: (1269, -1230, 61)
Found scanner 4 from 16: (2343, -1304, 29)
Found scanner 6 from 4: (2462, -1251, -1106)
Found scanner 1 from 6: (3537, -1150, -1217)
Found scanner 10 from 6: (2380, -1267, -2303)
Found scanner 19 from 6: (2287, -2446, -1197)
Found scanner 11 from 19: (1109, -2459, -1128)
Found scanner 5 from 11: (83, -2435, -1213)
Found scanner 2 from 5: (-43, -2452, -2367)
Found scanner 15 from 2: (66, -2393, -3614)
Found scanner 3 from 15: (1139, -2459, -3674)
Found scanner 8 from 15: (78, -3586, -3635)
Found scanner 12 from 8: (-1157, -3702, -3631)
Found scanner 18 from 2: (-76, -3579, -2294)
Found scanner 20 from 6: (1270, -1220, -1133)
Found scanner 21 from 5: (-1264, -2373, -1181)
Found scanner 23 from 2: (1231, -2497, -2442)
Found scanner 24 from 11: (1198, -3696, -1197)
Found scann

## Part 2

In [13]:
def manhatten_dist(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1]) + abs(p1[2] - p2[2])

dists = []
for i in range(len(scanner_positions)):
    for j in range(len(scanner_positions)):
        dists.append(manhatten_dist(scanner_positions[i], scanner_positions[j]))
max(dists)

9764